# Format_Data <a class="tocSkip">

In [76]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
import sys
import re
from pathlib import Path
import pandas as pd
import emoji
import neologdn

sys.path.append('../bin')
import myutil
import dataset
from dataset import Holomem

prj_path = myutil.PROJECT_PATH
prj_path

PosixPath('/Users/nyu/gdrive/learn/hololi2vec')

<h1>目次<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#取得ツイート確認" data-toc-modified-id="取得ツイート確認-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>取得ツイート確認</a></span></li><li><span><a href="#RT削除" data-toc-modified-id="RT削除-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>RT削除</a></span></li><li><span><a href="#URL除去" data-toc-modified-id="URL除去-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>URL除去</a></span></li><li><span><a href="#絵文字除去" data-toc-modified-id="絵文字除去-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>絵文字除去</a></span></li><li><span><a href="#正規化" data-toc-modified-id="正規化-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>正規化</a></span></li><li><span><a href="#リプライ先（@ユーザ名）の除去" data-toc-modified-id="リプライ先（@ユーザ名）の除去-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>リプライ先（@ユーザ名）の除去</a></span></li><li><span><a href="#ハッシュタグ除去" data-toc-modified-id="ハッシュタグ除去-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>ハッシュタグ除去</a></span></li><li><span><a href="#改行除去" data-toc-modified-id="改行除去-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>改行除去</a></span></li><li><span><a href="#半角記号除去" data-toc-modified-id="半角記号除去-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>半角記号除去</a></span></li><li><span><a href="#全角記号除去" data-toc-modified-id="全角記号除去-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>全角記号除去</a></span></li><li><span><a href="#数字除去" data-toc-modified-id="数字除去-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>数字除去</a></span></li><li><span><a href="#除草" data-toc-modified-id="除草-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>除草</a></span></li><li><span><a href="#特殊記号除去" data-toc-modified-id="特殊記号除去-13"><span class="toc-item-num">13&nbsp;&nbsp;</span>特殊記号除去</a></span></li><li><span><a href="#連続半角スペースをまとめる" data-toc-modified-id="連続半角スペースをまとめる-14"><span class="toc-item-num">14&nbsp;&nbsp;</span>連続半角スペースをまとめる</a></span></li><li><span><a href="#内容のないツイートを削除" data-toc-modified-id="内容のないツイートを削除-15"><span class="toc-item-num">15&nbsp;&nbsp;</span>内容のないツイートを削除</a></span></li><li><span><a href="#ツイートテキスト整形確認" data-toc-modified-id="ツイートテキスト整形確認-16"><span class="toc-item-num">16&nbsp;&nbsp;</span>ツイートテキスト整形確認</a></span></li><li><span><a href="#整形本番" data-toc-modified-id="整形本番-17"><span class="toc-item-num">17&nbsp;&nbsp;</span>整形本番</a></span></li></ul></div>

# 取得ツイート確認
`01_Gather_Data`で取得したシオンちゃんの最新3200ツイート（RT含む）から整形方針を考える

In [3]:
shion = Holomem(userId='murasakishionch', n_tweet=3200, load=True, verbose=False, preview=False)

In [ ]:
tweets = shion.df
tweets.head(50)

text列について，上記出力結果より以下の処理が必要と考える
- RTの削除
- リプライ先（@ユーザ名）の除去
- 絵文字・特殊記号の除去
- 伸ばし棒・三点リーダ・！・？などの感情的表現の除去
- URL

など...

整形結果を見ながら他にも施すべき処理がないか見ていく

**参考**
- https://ohke.hateblo.jp/entry/2019/02/09/141500
- https://qiita.com/Hironsan/items/2466fe0f344115aff177

# RT削除
- RTはテキストが必ず`RT`から始まるので，最初の2文字を見て判断すればよい
- この方法はただのツイートの最初の2文字が`RT`のときも削除されることが見込まれるが，そのようなケースはほとんど存在しないことが予想されるので無視する（誤ってRTとして削除されるのはやむを得ない）

In [ ]:
tweets = tweets[tweets['text'].str[:2] != 'RT'].copy()
tweets.head(20)

In [6]:
tweets.shape

(2635, 3)

RTは565/3200ツイートだった

# URL除去
- 自然言語から外れた要素なので除去する
- 参考: https://ohke.hateblo.jp/entry/2019/02/09/141500
- ↑「URLの除去」を参照

In [ ]:
sample_tweet = tweets['text'][2]
sample_tweet

In [ ]:
re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', sample_tweet)

画像付きツイートには`https://t.co/*******`というリンクが必ず末尾に付いているので除去すべき

In [ ]:
tweets['text'] = tweets['text'].map(lambda x: re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', x)).copy()
tweets.head(20)

# 絵文字除去
- `emoji`というライブラリを活用して除外できるらしい
- 参考: https://ohke.hateblo.jp/entry/2019/02/09/141500
- ↑「絵文字の除去 (emoji)」を参照

```
pip install emoji==1.2.0
```

In [ ]:
sample_tweet = tweets['text'][2]
sample_tweet

In [ ]:
''.join(['' if c in emoji.UNICODE_EMOJI else c for c in sample_tweet])

？？？？？？？？

In [12]:
emoji.UNICODE_EMOJI.keys()

dict_keys(['en', 'es', 'pt', 'it'])

各絵文字に4ヶ国語の説明が付与された，2次元の辞書になっていた（ここ最近の仕様変更？）

In [13]:
for k in emoji.UNICODE_EMOJI.keys():
    print(emoji.UNICODE_EMOJI[k]['💘'])

:heart_with_arrow:
:corazón_con_flecha:
:coração_com_flecha:
:cuore_con_freccia:


In [ ]:
''.join(['' if c in emoji.UNICODE_EMOJI['en'] else c for c in sample_tweet])

無事消えた

ちなみに，`emoji.UNICODE_EMOJI['en']`は以下のような構造になっている．

In [18]:
 [(k, v) for k, v in emoji.UNICODE_EMOJI['en'].items()][:5]

[('🥇', ':1st_place_medal:'),
 ('🥈', ':2nd_place_medal:'),
 ('🥉', ':3rd_place_medal:'),
 ('🆎', ':AB_button_(blood_type):'),
 ('🏧', ':ATM_sign:')]

In [ ]:
tweets['text'] = tweets['text'].map(lambda x: ''.join(['' if c in emoji.UNICODE_EMOJI['en'] else c for c in x])).copy()
tweets.head(20)

`▹▸`は絵文字かと思いきや特殊記号で除去できてなかった（後ほど除去する予定）

# 正規化
- `neologdn`というライブラリを活用して除外できるらしい
- 参考: https://ohke.hateblo.jp/entry/2019/02/09/141500
- ↑「全角・半角の統一と重ね表現の除去 (neologdn)」を参照

### neologdn <a class="tocSkip">
- https://github.com/ikegami-yukino/neologdn
- `mecab-neologdn`による日本語テキスト正規化ツール
- 正規化は[`neologd`のルール](https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja)に基づいている
- 正規化: 全角半角や重ね表現の除去など，細かい表現を統一させること
    
```
pip install neologdn==0.4
```

In [ ]:
sample_tweet = tweets['text'][6]
sample_tweet

In [ ]:
neologdn.normalize(sample_tweet)

*すごい*

`〜`が除去され，`！`(全角)が`!`（半角）になっている

In [ ]:
tweets['text'] = tweets['text'].map(lambda x: neologdn.normalize(x)).copy()
tweets.head(20)

まだ不要と思われる要素が残っているが，最初より大分すっきりした印象

# リプライ先（@ユーザ名）の除去
次の2パターンがある
- ただのリプ
```
@aiueo どうも
```

- サンクスツイート（勝手に名付けました）
```
サムネイラスト！ @ aiueo
```
↑相手に通知がいかないorユーザページへのリンクにならないよう， `@`とユーザ名の間に半角スペースを空けている

- つまり，`@`とユーザ名（アルファベットと数字で構成）の半角スペースが0/1回のパターンを除去すれば良い
- `@`とユーザ名の間が全角スペースになっている可能性もあるが，先の`neologdn`による正規化で[全角スペースはすべて半角スペースに置換されている](https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja#%E5%85%A8%E8%A7%92%E3%82%B9%E3%83%9A%E3%83%BC%E3%82%B9%E3%81%AF%E5%8D%8A%E8%A7%92%E3%82%B9%E3%83%9A%E3%83%BC%E3%82%B9%E3%81%AB%E7%BD%AE%E6%8F%9B)ので，上のルールで問題ない
- `@`とユーザ名は自然言語から外れた要素なので除去すべき

In [ ]:
sample_tweet_1, sample_tweet_2 = tweets['text'][1], tweets['text'][11]
sample_tweet_1, sample_tweet_2

In [ ]:
print(re.sub(r'@[ ]?[a-zA-z0-9]+', '', sample_tweet_1))
print(re.sub(r'@[ ]?[a-zA-z0-9]+', '', sample_tweet_2))

いずれもリプライ先の部分が除去できていることがわかる

In [ ]:
tweets['text'] = tweets['text'].map(lambda x: re.sub(r'@[ ]?[a-zA-z0-9]+', '', x)).copy()
tweets.head(20)

#  ハッシュタグ除去
- ハッシュタグは文章のパーツにあたらないため除去する
- 参考: https://qiita.com/corin8823/items/75309761833d823cac6f

In [ ]:
sample_tweet = tweets['text'][30]
sample_tweet

- `#`が全角の可能性もあるが，[neologdnによって半角に置換されている](https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja#%E4%BB%A5%E4%B8%8B%E3%81%AE%E5%85%A8%E8%A7%92%E8%A8%98%E5%8F%B7%E3%81%AF%E5%8D%8A%E8%A7%92%E8%A8%98%E5%8F%B7%E3%81%AB%E7%BD%AE%E6%8F%9B)ので，以下の正規表現で問題ない
- 全角英数字も同様に[neologdnで半角に置換されている](https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja#%E5%85%A8%E8%A7%92%E8%8B%B1%E6%95%B0%E5%AD%97%E3%81%AF%E5%8D%8A%E8%A7%92%E3%81%AB%E7%BD%AE%E6%8F%9B)ので問題ない

In [ ]:
re.sub(r'[#][A-Za-z一-鿆0-9ぁ-ヶｦ-ﾟー]+', '', sample_tweet)

[Bloom,](https://bloom.hololive.tv/)のハッシュタグを除去できた（※このノートをGitHubのプッシュする際，出力されたツイート内容はすべて消しています）

In [ ]:
tweets['text'] = tweets['text'].map(lambda x: re.sub(r'[#][A-Za-z一-鿆0-9ぁ-ヶｦ-ﾟー]+', '', x)).copy()
tweets.head(20)

# 改行除去
- 自然言語から外れた要素なので除去する
- いずれも半角スペースに置き換える

In [ ]:
sample_tweet = tweets['text'][6]
sample_tweet

In [ ]:
re.sub(r'[\n]', ' ', sample_tweet)

すべて消えていることを確認

In [ ]:
tweets['text'] = tweets['text'].map(lambda x: re.sub(r'[\n]', ' ', x)).copy()
tweets.head(20)

# 半角記号除去
- 半角記号リストは以下で示されているものを参考（一部追加）
- https://qiita.com/waterame/items/28f52b8f4ffff81d658b
- 半角スペースに置換する

In [ ]:
sample_tweet = tweets['text'][6]
sample_tweet

In [ ]:
re.sub(r'[!\"\#$%&\'()*+,-./:;<=>?@[\]^_`{|}~]', ' ', sample_tweet)

In [ ]:
tweets['text'] = tweets['text'].map(lambda x: re.sub(r'[!\"\#$%&\'()*+,-./:;<=>?@[\]^_`{|}~\\]', ' ', x)).copy()
tweets.head(20)

# 全角記号除去
- `neologdn`は[ある程度の全角記号を半角記号に置換している](https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja#%E4%BB%A5%E4%B8%8B%E3%81%AE%E5%85%A8%E8%A7%92%E8%A8%98%E5%8F%B7%E3%81%AF%E5%8D%8A%E8%A7%92%E8%A8%98%E5%8F%B7%E3%81%AB%E7%BD%AE%E6%8F%9B)ので，先の半角記号除去で消えているものもあるが，一部全角記号は置換されず全角のまま残っており除去できていない
- 手動設定で可能な限り除去する

In [ ]:
sample_tweet = tweets['text'][2]
sample_tweet

In [ ]:
re.sub(r'[・〜＋｜’；？：＞＜」「（）％＄＃＠＆＾＊！？【】『』＼／…○△□●▲■▼▶◀▽★☆※‥]', '', sample_tweet)

↑全角記号除去が素晴らしく貢献した例

In [ ]:
tweets['text'] = tweets['text'].map(lambda x: re.sub(r'[・〜＋｜’；：＞＜」「（）％＄＃＠＆＾＊！？【】『』［］＼／…○△□●▲■▼▶◀▽★☆※‥]', '', x)).copy()
tweets.head(20)

# 数字除去
- 見た感じ数字も不要な気がしてきた
    - 出現回数多くなりすぎる
    - 半角記号消した結果意味不明なものになっているケースがある
- [全角数字は`neologdn`で半角に置換されている](https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja#%E5%85%A8%E8%A7%92%E8%8B%B1%E6%95%B0%E5%AD%97%E3%81%AF%E5%8D%8A%E8%A7%92%E3%81%AB%E7%BD%AE%E6%8F%9B)ため，半角数字のみ除去すれば良い

In [ ]:
sample_tweet = tweets['text'][20]
sample_tweet

In [ ]:
re.sub(r'[0-9]', '', sample_tweet)

半角記号除去で`:`を消した結果，無意味になっていた時刻表記の残骸を消せた

In [ ]:
tweets['text'] = tweets['text'].map(lambda x: re.sub(r'[0-9]', '', x)).copy()
tweets.head(20)

# 除草
-「草」は除外しないが，「www」は除外する
- 単芝は除外しない（wを含む単語が崩壊するため）
- 全角`ｗ`は[neologdnで半角に置換されている](https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja#%E5%85%A8%E8%A7%92%E8%8B%B1%E6%95%B0%E5%AD%97%E3%81%AF%E5%8D%8A%E8%A7%92%E3%81%AB%E7%BD%AE%E6%8F%9B)ので正規表現に含める必要はない

In [44]:
sample_tweet = 'えwwむりｗあwww'
sample_tweet

'えwwむりｗあwww'

In [45]:
re.sub(r'[w]{2,}', ' ', sample_tweet)

'え むりｗあ '

In [ ]:
tweets['text'] = tweets['text'].map(lambda x: re.sub(r'[w]{2,}', ' ', x)).copy()
tweets.head(20)

# 特殊記号除去
- 多くのホロメンのツイートをチェックして特殊な記号リストを用意
- 予想していたことだが，かなり特殊な記号（どこの言語やねんってやつ）を使った顔文字が多かった
- これでも完全に除去できていないが，キリがないので諦める
- 今更だが，Unicode範囲指定の方が良かったかもしれない（手動で済むと思いきやめちゃくちゃ多種類だった）
- あとのモデル学習結果があまりにも酷い場合は再整形する

In [66]:
sample_tweet = r'ก˶᷇ ᷆˵ᯅʅʃ▓ु▷§٩งᐠ•๑ว≡￣⃕シᐟ𓏸◝ཫᆺ́ㅅ♡̩˘◥￢ᔦ✿﹀◦⌓┃₍₍⁾⁾  ृ๛ ͟͟͞ᔨ⸜ㅂ◤꒳。オ◟ﾟ｀ʖ└ᴗ´⋆˚ଘ⸝╹│Ꙭ̮𓂂▹▸°↝̀͜੭ンㄘζو̑̑߹〃³¦༥˙ꇤ꜄꜆ ⃔зੈ॑₊ʘ ωᗜ⊂‧ᐛ⌒♆❛ᵕ✩♪◡▀̿ヾ𖥦ᵒ̴̶̷✧ˆˊˋ'
sample_tweet

'ก˶᷇ ᷆˵ᯅʅʃ▓ु▷§٩งᐠ•๑ว≡￣⃕シᐟ𓏸◝ཫᆺ́ㅅ♡̩˘◥￢ᔦ✿﹀◦⌓┃₍₍⁾⁾  ृ๛ ͟͟͞ᔨ⸜ㅂ◤꒳。オ◟ﾟ｀ʖ└ᴗ´⋆˚ଘ⸝╹│Ꙭ̮𓂂▹▸°↝̀͜੭ンㄘζو̑̑߹〃³¦༥˙ꇤ꜄꜆ ⃔зੈ॑₊ʘ ωᗜ⊂‧ᐛ⌒♆❛ᵕ✩♪◡▀̿ヾ𖥦ᵒ̴̶̷✧ˆˊˋ'

In [67]:
re.sub(r'[ก˶᷇ ᷆˵ᯅʅʃ▓ु▷§٩งᐠ•๑ว≡￣⃕ᐟ𓏸◝ཫᆺ́ㅅ♡̩˘◥￢ᔦ✿﹀◦⌓┃₍₍⁾⁾  ृ๛ ͟͟͞ᔨ⸜ㅂ◤꒳。◟ﾟ｀ʖ└ᴗ´⋆˚ଘ⸝╹│Ꙭ̮𓂂▹▸°↝̀͜੭ㄘζو̑̑߹〃³¦༥˙ꇤ꜄꜆ ⃔зੈ॑₊ʘ ωᗜ⊂‧ᐛ⌒♆❛ᵕ✩♪◡▀̿ヾ𖥦ᵒ̴̶̷✧ˆˊˋ]', ' ', sample_tweet)

'                      シ                                    オ                       ン                                              '

In [ ]:
tweets['text'] = tweets['text'].map(lambda x: re.sub(r'[ก˶᷇ ᷆˵ᯅʅʃ▓ु▷§٩งᐠ•๑ว≡￣⃕ᐟ𓏸◝ཫᆺ́ㅅ♡̩˘◥￢ᔦ✿﹀◦⌓┃₍₍⁾⁾  ृ๛ ͟͟͞ᔨ⸜ㅂ◤꒳。◟ﾟ｀ʖ└ᴗ´⋆˚ଘ⸝╹│Ꙭ̮𓂂▹▸°↝̀͜੭ㄘζو̑̑߹〃³¦༥˙ꇤ꜄꜆ ⃔зੈ॑₊ʘ ωᗜ⊂‧ᐛ⌒♆❛ᵕ✩♪◡▀̿ヾ𖥦ᵒ̴̶̷✧ˆˊˋ]', ' ', x)).copy()
tweets.head(20)

シオンちゃんの最新30ツイートに限っては特殊記号はすべて除去できている

# 連続半角スペースをまとめる
- [`neologdn`は1つ以上の半角スペースを1つの半角スペースに置換する](https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja#1%E3%81%A4%E4%BB%A5%E4%B8%8A%E3%81%AE%E5%8D%8A%E8%A7%92%E3%82%B9%E3%83%9A%E3%83%BC%E3%82%B9%E3%81%AF1%E3%81%A4%E3%81%AE%E5%8D%8A%E8%A7%92%E3%82%B9%E3%83%9A%E3%83%BC%E3%82%B9%E3%81%AB%E7%BD%AE%E6%8F%9B)が，`neologdn`による正規化のあとで行った処理の結果，新たに連続半角スペースが誕生している可能性があるため，改めてこの処理を行う
- [全角スペースは半角スペースに置換しており](https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja#%E5%85%A8%E8%A7%92%E3%82%B9%E3%83%9A%E3%83%BC%E3%82%B9%E3%81%AF%E5%8D%8A%E8%A7%92%E3%82%B9%E3%83%9A%E3%83%BC%E3%82%B9%E3%81%AB%E7%BD%AE%E6%8F%9B)，以後の処理で全角スペースへの置換処理は行っていないので正規表現には不要

In [51]:
sample_tweet = 'xxxxx     あ w x x  w'
sample_tweet

'xxxxx     あ w x x  w'

In [52]:
re.sub(r'[ ]+', ' ', sample_tweet)

'xxxxx あ w x x w'

In [ ]:
tweets['text'] = tweets['text'].map(lambda x: re.sub(r'[ ]+', ' ', x)).copy()
tweets.head(20)

# 内容のないツイートを削除
除去処理の結果内容が消滅したツイートは行ごと消す

In [ ]:
tweets = tweets[~tweets['text'].isin([' ', ''])]
tweets.head(20)

In [55]:
len(tweets)

2573

RT削除時点で2635/3200ツイート残っていたが，最終的に62ツイートさらに消えて，シオンちゃんのツイート数は2573になった

# ツイートテキスト整形確認
- かなり掃除できてきたので，他のホロメンでも同様の除外をしてみる
- RT除去は行自体削除するので関数適用前に実行する
- 内容のないツイートも行自体削除するが，これは関数適用後に実行する
- あまりに無視できない不要な文字があれば後ほど除外対象に加える（「13.特殊記号除去」はこの結果を見ながら行っている）

In [63]:
%autoreload

In [64]:
hololive_members = pd.read_csv(Path(prj_path, 'data/hololive_members.csv'))
hololive_members.head()

,twitter,name,generation
0,tokino_sora,ときのそら,0
1,robocosan,ロボ子さん,0
2,sakuramiko35,さくらみこ,0
3,suisei_hosimati,星街すいせい,0
4,yozoramel,夜空メル,1


In [ ]:
for userId in hololive_members['twitter']:
    holomem = Holomem(userId=userId, n_tweet=3200, load=True, verbose=False, preview=False)
    tweets = holomem.df.copy()
    latest_30_tweets = tweets[tweets['text'].str[:2] != 'RT'].head(30)
    
    latest_30_tweets['text'] = latest_30_tweets['text'].map(dataset.format_tweet_text).copy()
    latest_30_tweets = latest_30_tweets[~latest_30_tweets['text'].isin([' ', ''])]
    print(userId)
    print(latest_30_tweets['text'])
    print()

- 各ホロメンの最新30ツイートに限っては不要そうな文字はすべて除去できている（はず）
- これ以上精査してもキリがないので一旦整形処理はここで切り上げる

# 整形本番
- 上記for文の処理を`Holomem`クラスのインスタンスメソッド`Holomem.format_tweets_df()`とし，各ホロメンのツイートデータフレームを変換する
- 念のため（整形方針を立て直すなど），各ホロメンの変換前のデータフレーム（tsv形式ファイル）は`data/rt_true_3200`にコピーして残しておく

In [80]:
%%time
for userId in hololive_members['twitter']:
    holomem = Holomem(userId=userId, n_tweet=3200, load=True, verbose=False, preview=False)
    holomem.format_tweets_df()

../bin/dataset.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df_without_rt['text'] = tweets_df_without_rt['text'].map(format_tweet_text).copy()


CPU times: user 15.7 s, sys: 389 ms, total: 16.1 s
Wall time: 16.1 s


In [ ]:
for userId in hololive_members['twitter']:
    holomem = Holomem(userId=userId, n_tweet=3200, load=True, verbose=False, preview=False)
    
    print(userId)
    print(holomem.df['text'].head())
    print()